# German Text Summary with the tranlated CNN Daily Mail Dataset
and T5 from Huggingface Pytorch

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from pathlib import Path
import torch
import re
import time

In [2]:
BATCH_SIZE = 16

SHUFFEL_SIZE = 1024

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

learning_rate = 3e-5

EPOCHS = 5

log_interval = 200

In [3]:
print(device)

cuda:0


## Define Model

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)

task_specific_params = model.config.task_specific_params
if task_specific_params is not None:
    model.config.update(task_specific_params.get("summarization", {}))
    

optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate, weight_decay=0.0001)

In [5]:
model.config.prefix

'summarize: '

## Read the tranlated German Dataset

In [6]:
def get_translated_ds(name):
    article_path = "../data/%s/articles_german" % name
    highlights_path = "../data/%s/highlights_german" % name

    articles = [x.rstrip() for x in open(article_path).readlines()]
    highlights = [x.rstrip() for x in open(highlights_path).readlines()]
    
    assert len(articles) == len(highlights)
    return articles, highlights

In [7]:
train_x, train_y = get_translated_ds("train")
test_x, test_y = get_translated_ds("test")
val_x, val_y = get_translated_ds("val")

## Show some Dataset Example 

In [8]:
import numpy as np
for i in range(5):
    rand_int = np.random.randint(len(train_x))
    print("\n--------")
    print(train_x[rand_int][:1500])
    print()
    print(train_y[rand_int])


--------
94358; Los Angeles, Kalifornien (CNN) - Conan O 'Briens Rückkehr ins Fernsehen am Montagabend blieb nah am üblichen Late-Night-Talkshow-Format und -Set, aber er konnte sich einer Innovation rühmen. "Ich glaube, wir sind die erste Talkshow, die einen Blick auf den Ozean hat", sagte O 'Brien. "Man kann die Salzwasserluft riechen". O 'Brien und Ansager-Sidekick Andy Richter demonstrierten ein weiteres neues Hintergrundmerkmal - einen überdimensionalen Mond, den sie mit einer Fernbedienung über den Himmel bewegen können. Exklusiv: Sehen Sie sich Conans Opener an. Ansonsten gibt es in der einstündigen TBS-Show Schreibtisch, Couch, Sidekick, Band und einen Gast, der für einen Film wirbt, der dem Late-Night-Publikum seit Jahrzehnten vertraut ist. O 'Briens notwendiger Eröffnungsmonolog machte deutlich, dass "gefeuerte" Witze für Conan das sind, was Scheidungswitze für Johnny Carson waren. "Die Leute fragen mich, warum ich die Serie" Conan "genannt habe", sagte er. "Ich habe es so ge

In [9]:
import numpy as np
for i in range(5):
    rand_int = np.random.randint(len(test_x))
    print("\n--------")
    print(test_x[rand_int][:1500])
    print()
    print(test_y[rand_int])


--------
4381; Eine Frau aus Arizona erlebte eine frühe Osterüberraschung, als sie bei einer Untersuchung im letzten Monat das Bild von Jesus Christus in ihrem Zahnröntgenbild sah. Kym Ackerman sah den Sohn Gottes im Röntgenbild eines ihrer linken Backenzähne, als sie am 25. März zu einer Zahnuntersuchung nach Flagstaff ging. Ackermann, 32, entdeckte das Bild und zeigte es sowohl dem Zahnarzt als auch dem Hygieniker. Blättern Sie nach unten für ein Video. Kym Ackerman sah Jesus auf einem Röntgenbild ihrer Backenzähne, als sie sich in Flagstaff, Arizona, einer zahnärztlichen Untersuchung unterzog. Ackerman, 32, plant, das Röntgenbild zu rahmen und den speziellen Backenzahn und ihren Mund in Zukunft frei von Hohlräumen zu halten. Obwohl der Hygieniker der geheiligten Form zustimmte, war der Zahnarzt wesentlich weniger aufgeregt und "begann, auf meine Zähne zu schauen", erzählte Ackerman der Huffington Post. Sie sagte: "Bei meinem Zahnarzt zeigt der Computer die Röntgenbilder an, sobald 

## Define Pytorch Dataset

In [10]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, articles, highlights):
        self.x = articles
        self.y = highlights
        
    def __getitem__(self, index):
        x = tokenizer.encode_plus(model.config.prefix + self.transfrom(self.x[index][3:]), max_length=512, return_tensors="pt", pad_to_max_length=True)
        y = tokenizer.encode(self.transfrom(self.y[index][3:]), max_length=150, return_tensors="pt", pad_to_max_length=True)
        return x['input_ids'].view(-1), x['attention_mask'].view(-1), y.view(-1)
    
    @staticmethod
    def transfrom(x):
        x = x.lower()
        x = re.sub("'(.*)'", r"\1", x)
        return x
    
    def __len__(self):
        return len(self.x)

In [11]:
train_ds = MyDataset(train_x, train_y) 
val_ds = MyDataset(val_x, val_y)
test_ds = MyDataset(test_x, test_y)

In [12]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE)

In [13]:
x, x_mask, y = next(iter(val_loader))
x.shape, x_mask.shape, y.shape

(torch.Size([16, 512]), torch.Size([16, 512]), torch.Size([16, 150]))

In [14]:
start_time = time.time()
for i in range(100):
    x, x_mask, y = next(iter(val_loader))
elapsed = time.time() - start_time
elapsed

8.418677568435669

## Define Step function

In [13]:
pad_token_id = tokenizer.pad_token_id
def step(inputs_ids, attention_mask, y):
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone()
    lm_labels[y[:, 1:] == pad_token_id] = -100
    output = model(inputs_ids, attention_mask=attention_mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
    return output[0] # loss

## Train

In [14]:
train_loss = []
val_loss = []
for epoch in range(EPOCHS):
    model.train() 
    start_time = time.time()
    for i, (inputs_ids, attention_mask, y) in enumerate(train_loader):
        inputs_ids = inputs_ids.to(device)
        attention_mask = attention_mask.to(device)
        y = y.to(device)
        
        
        optimizer.zero_grad()
        loss = step(inputs_ids, attention_mask, y)
        train_loss.append(loss.item())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
            
        if (i + 1) % log_interval == 0:
            with torch.no_grad():
                x, x_mask, y = next(iter(val_loader))
                x = x.to(device)
                x_mask = x_mask.to(device)
                y = y.to(device)
                
                v_loss = step(x, x_mask, y)
                v_loss = v_loss.item()
                
                
                elapsed = time.time() - start_time
                print('| epoch {:3d} | [{:5d}/{:5d}] | '
                  'ms/batch {:5.2f} | '
                  'loss {:5.2f} | val loss {:5.2f}'.format(
                    epoch, i, len(train_loader),
                    elapsed * 1000 / log_interval,
                    loss.item(), v_loss))
                start_time = time.time()
                val_loss.append(v_loss)
                
                

| epoch   0 | [  199/17945] | ms/batch 403.61 | loss  3.03 | val loss  2.91
| epoch   0 | [  399/17945] | ms/batch 399.79 | loss  2.92 | val loss  2.81
| epoch   0 | [  599/17945] | ms/batch 398.31 | loss  3.00 | val loss  2.91
| epoch   0 | [  799/17945] | ms/batch 399.74 | loss  2.58 | val loss  2.80
| epoch   0 | [  999/17945] | ms/batch 402.75 | loss  2.81 | val loss  2.83
| epoch   0 | [ 1199/17945] | ms/batch 402.89 | loss  2.48 | val loss  2.86
| epoch   0 | [ 1399/17945] | ms/batch 403.91 | loss  2.44 | val loss  2.83
| epoch   0 | [ 1599/17945] | ms/batch 408.64 | loss  2.60 | val loss  2.90
| epoch   0 | [ 1799/17945] | ms/batch 402.08 | loss  2.49 | val loss  2.88
| epoch   0 | [ 1999/17945] | ms/batch 407.34 | loss  2.53 | val loss  2.90
| epoch   0 | [ 2199/17945] | ms/batch 404.00 | loss  2.33 | val loss  2.86
| epoch   0 | [ 2399/17945] | ms/batch 407.81 | loss  2.76 | val loss  2.93
| epoch   0 | [ 2599/17945] | ms/batch 405.31 | loss  2.36 | val loss  2.89
| epoch   0 

| epoch   1 | [ 3999/17945] | ms/batch 410.35 | loss  2.69 | val loss  3.93
| epoch   1 | [ 4199/17945] | ms/batch 410.20 | loss  2.64 | val loss  3.83
| epoch   1 | [ 4399/17945] | ms/batch 391.29 | loss  2.78 | val loss  3.89
| epoch   1 | [ 4599/17945] | ms/batch 381.55 | loss  2.71 | val loss  3.97
| epoch   1 | [ 4799/17945] | ms/batch 381.39 | loss  2.67 | val loss  3.92
| epoch   1 | [ 4999/17945] | ms/batch 384.19 | loss  2.77 | val loss  3.91
| epoch   1 | [ 5199/17945] | ms/batch 383.98 | loss  2.94 | val loss  3.96
| epoch   1 | [ 5399/17945] | ms/batch 382.51 | loss  2.65 | val loss  3.98
| epoch   1 | [ 5599/17945] | ms/batch 384.37 | loss  2.49 | val loss  3.91
| epoch   1 | [ 5799/17945] | ms/batch 385.36 | loss  2.73 | val loss  3.96
| epoch   1 | [ 5999/17945] | ms/batch 385.77 | loss  3.04 | val loss  3.95
| epoch   1 | [ 6199/17945] | ms/batch 388.01 | loss  2.53 | val loss  3.98
| epoch   1 | [ 6399/17945] | ms/batch 384.48 | loss  2.75 | val loss  3.98
| epoch   1 

| epoch   2 | [ 7799/17945] | ms/batch 384.85 | loss  3.18 | val loss  3.99
| epoch   2 | [ 7999/17945] | ms/batch 384.26 | loss  2.99 | val loss  3.93
| epoch   2 | [ 8199/17945] | ms/batch 387.59 | loss  3.04 | val loss  4.00
| epoch   2 | [ 8399/17945] | ms/batch 383.27 | loss  3.22 | val loss  3.94
| epoch   2 | [ 8599/17945] | ms/batch 385.03 | loss  2.96 | val loss  3.97
| epoch   2 | [ 8799/17945] | ms/batch 383.46 | loss  2.61 | val loss  3.98
| epoch   2 | [ 8999/17945] | ms/batch 383.67 | loss  2.96 | val loss  4.01
| epoch   2 | [ 9199/17945] | ms/batch 383.93 | loss  3.05 | val loss  3.93
| epoch   2 | [ 9399/17945] | ms/batch 383.23 | loss  2.80 | val loss  4.06
| epoch   2 | [ 9599/17945] | ms/batch 386.59 | loss  2.79 | val loss  4.00
| epoch   2 | [ 9799/17945] | ms/batch 385.45 | loss  2.87 | val loss  3.89
| epoch   2 | [ 9999/17945] | ms/batch 387.14 | loss  2.82 | val loss  3.89
| epoch   2 | [10199/17945] | ms/batch 387.77 | loss  3.04 | val loss  3.95
| epoch   2 

KeyboardInterrupt: 

## Evaluate

In [15]:
from rouge_score import rouge_scorer
from rouge_score import scoring

class RougeScore:
    '''
    mostly from https://github.com/google-research/text-to-text-transfer-transformer/blob/master/t5/evaluation/metrics.py 
    '''
    
    def __init__(self, score_keys=None)-> None:
        super().__init__()
        if score_keys is None:  
            self.score_keys = ["rouge1", "rouge2", "rougeLsum"]
        
        self.scorer = rouge_scorer.RougeScorer(self.score_keys)
        self.aggregator = scoring.BootstrapAggregator()
        
        
    @staticmethod
    def prepare_summary(summary):
            # Make sure the summary is not bytes-type
            # Add newlines between sentences so that rougeLsum is computed correctly.
            summary = summary.replace(" . ", " .\n")
            return summary
    
    def __call__(self, target, prediction):
        """Computes rouge score.''
        Args:
        targets: string
        predictions: string
        """

        target = self.prepare_summary(target)
        prediction = self.prepare_summary(prediction)
        
        self.aggregator.add_scores(self.scorer.score(target=target, prediction=prediction))

        return 
    
    def reset_states(self):
        self.rouge_list = []

    def result(self):
        result = self.aggregator.aggregate()
        
        for key in self.score_keys:
            score_text = "%s = %.2f, 95%% confidence [%.2f, %.2f]"%(
                key,
                result[key].mid.fmeasure*100,
                result[key].low.fmeasure*100,
                result[key].high.fmeasure*100
            )
            print(score_text)
        
        return {key: result[key].mid.fmeasure*100 for key in self.score_keys}

In [16]:
rouge_score = RougeScore()
predictions = []
for i, (input_ids, attention_mask, y) in enumerate(test_loader):
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    y = y.to(device)
        
    summaries = model.generate(input_ids=input_ids, attention_mask=attention_mask)
    pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
    real = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in y]
    for pred_sent, real_sent in zip(pred, real):
        rouge_score(pred_sent, real_sent)
        predictions.append(str("pred sentence: " + pred_sent + "\n\n real sentence: " + real_sent))
    if i > 40:
        break
    
rouge_score.result()

rouge1 = 26.67, 95% confidence [26.08, 27.29]
rouge2 = 8.33, 95% confidence [7.86, 8.73]
rougeLsum = 15.57, 95% confidence [15.18, 15.92]


{'rouge1': 26.668475437578245,
 'rouge2': 8.333081382416019,
 'rougeLsum': 15.566398570250495}

In [17]:
for pred in predictions[:10]:
    print("------")
    print(pred)
    print("------")    

------
pred sentence: flugzeuge, ob ein derart vollvollgepacke immer kleiner werden. die us-verbraucherberatungsgruppe, die die gesundheit und sicherheit gefährdet? sie sagt, dass der schrumpfende platz für tiere im platz in sicherheit. mehr als die armlehne, platzmangel im verkehrsministerium. cynthia corbertt, dass er eine behandlung der passagiere ein standard, der bei der faa durchgeführt werden werden können. viele zoll platz fehlen fehlen, dass die fluggesellschaften mit einem sitz bis bis zum gleichen flugflugzeugen führen, während der welt. es sagte, dass eine gesundheit gesundheit für sicherheitsgruppe ernsterstert. er sagt, er fußußtritten. viele economy-sitze 

 real sentence: experten bezweifeln, ob überfüllte flugzeuge passagiere gefährden. die us-verbraucherberatungsgruppe sagt, dass mindestabstände vorgeschrieben werden müssen. sicherheitstests in flugzeugen mit mehr beinfreiheit als von fluggesellschaften angeboten, werden durchgeführt.
------
------
pred sentence: rahu

In [18]:
result_path = "../data/result_german.txt"
for pred in predictions:
    with open(result_path, "a") as file:
        file.write(pred + "\n")
